# Validate and prepare data for BIOMASS widget  

Data model:  
```location_id``` [str]  
```biomass_density_class``` [str] (category) and average  
```value``` [number]  
```year``` [int]   only 2020 for now

In [2]:
import pandas as pd
import numpy as np
import geopandas as gpd
import pandera as pa
from pandera.typing import Series
import requests
from pathlib import Path
import os
import logging

## Load data  
Data is stored in the S3 bucket of the project

### Country data

In [3]:
file_name = "gmw_v3_agb_summary_bounds.xlsx"
data_sheet = 'gmw_agb_stats'

bucket_url = "https://storage.googleapis.com/mangrove_atlas/widget_data/"
data_url = f'{bucket_url}{file_name}'

mang_df = pd.read_excel(data_url, sheet_name=data_sheet)
mang_df.head()

,Unnamed: 0,Country,Country_Code,0-50,50-100,100-150,150-250,250-1500
0,0,Angola,AGO,0.371058,0.232452,0.114814,0.10162,0.180056
1,1,Anguilla,AIA,1.000000,0.000000,0.000000,0.00000,0.000000
2,2,United Arab Emirates,ARE,1.000000,0.000000,0.000000,0.00000,0.000000
3,3,Bahrain,BHR,0.883077,0.116923,0.000000,0.00000,0.000000
4,4,Bahamas,BHS,1.000000,0.000000,0.000000,0.00000,0.000000


### WDPA data

In [4]:
file_name = "gmw_v3_agb_protect_area_bounds.xlsx"
data_sheet = 'gmw_agb_stats'

bucket_url = "https://storage.googleapis.com/mangrove_atlas/widget_data/"
data_url = f'{bucket_url}{file_name}'
wdpa_df = pd.read_excel(data_url, sheet_name=data_sheet)
wdpa_df.head()

,Unnamed: 0,WDPAID,0-50,50-100,100-150,150-250,250-1500
0,0,24,1.000000,0.000000,0.000000,0.000000,0.0
1,1,42,0.054312,0.172469,0.226589,0.546630,0.0
2,2,48,0.672616,0.239822,0.066369,0.021194,0.0
3,3,57,0.185951,0.154087,0.173803,0.486159,0.0
4,4,61,0.488173,0.390986,0.107780,0.013061,0.0


### Country average and total data

In [6]:
file_name = "gmw_v3_srtm_agb_stats.xlsx"
data_sheet = 'gmw_agb_stats'

bucket_url = "https://storage.googleapis.com/mangrove_atlas/widget_data/"
data_url = f'{bucket_url}{file_name}'
mang_stats_df = pd.read_excel(data_url, sheet_name=data_sheet)
mang_stats_df.head()

,Country,Country_Code,1996_agb_tot,1996_agb_avg,2007_agb_tot,2007_agb_avg,2008_agb_tot,2008_agb_avg,2009_agb_tot,2009_agb_avg,...,2016_agb_tot,2016_agb_avg,2017_agb_tot,2017_agb_avg,2018_agb_tot,2018_agb_avg,2019_agb_tot,2019_agb_avg,2020_agb_tot,2020_agb_avg
0,Angola,AGO,3.765229e+06,128.273608,3.759281e+06,130.076244,3.753697e+06,130.735320,3.754511e+06,130.576163,...,3.724652e+06,131.385108,3.730111e+06,131.543453,3.717747e+06,132.345317,3.718043e+06,131.830731,3.710737e+06,132.098473
1,Anguilla,AIA,2.064141e+01,5.076736,1.064498e+01,3.927247,7.904306e+00,3.240185,1.362714e+01,5.200811,...,2.976402e+01,6.215414,2.093303e+01,4.542735,1.509120e+01,4.175640,1.486861e+01,4.701768,1.530619e+01,4.705687
2,United Arab Emirates,ARE,5.924082e+04,7.808334,6.447833e+04,8.021529,6.877155e+04,8.000972,6.498133e+04,7.774841,...,5.521319e+04,7.811153,5.564499e+04,7.688939,5.612792e+04,7.625809,5.686886e+04,7.683620,5.707837e+04,7.731556
3,Bahrain,BHR,6.226990e+02,9.714159,6.468289e+02,7.426509,6.503066e+02,7.083951,6.526945e+02,6.986415,...,6.015747e+02,10.365874,6.015747e+02,10.305166,6.025237e+02,10.042014,6.025237e+02,10.187130,6.241641e+02,10.522623
4,Bahamas,BHS,1.958005e+06,11.552526,1.832593e+06,11.526773,1.755861e+06,11.599976,1.753588e+06,11.602877,...,1.710169e+06,11.900106,1.715869e+06,11.856941,1.719506e+06,11.840324,1.745661e+06,11.860113,1.788835e+06,11.907751


## Clean and prepare data

In [7]:
mang_df.drop(columns=['Unnamed: 0','Country'], inplace=True)

mang_df.head()

,Country_Code,0-50,50-100,100-150,150-250,250-1500
0,AGO,0.371058,0.232452,0.114814,0.10162,0.180056
1,AIA,1.000000,0.000000,0.000000,0.00000,0.000000
2,ARE,1.000000,0.000000,0.000000,0.00000,0.000000
3,BHR,0.883077,0.116923,0.000000,0.00000,0.000000
4,BHS,1.000000,0.000000,0.000000,0.00000,0.000000


In [8]:
mang_stats_df = mang_stats_df[['Country_Code', '2020_agb_tot','2020_agb_avg']]
mang_stats_df.head()

,Country_Code,2020_agb_tot,2020_agb_avg
0,AGO,3.710737e+06,132.098473
1,AIA,1.530619e+01,4.705687
2,ARE,5.707837e+04,7.731556
3,BHR,6.241641e+02,10.522623
4,BHS,1.788835e+06,11.907751


Get data in long format, with years as variable field

In [9]:
mang_df_long = mang_df.melt(id_vars='Country_Code')
mang_df_long = mang_df_long[~mang_df_long.Country_Code.isna()]
mang_df_long

,Country_Code,variable,value
0,AGO,0-50,0.371058
1,AIA,0-50,1.000000
2,ARE,0-50,1.000000
3,BHR,0-50,0.883077
4,BHS,0-50,1.000000
...,...,...,...
605,TON,250-1500,0.000000
606,TTO,250-1500,0.000000
607,VGB,250-1500,0.000000
608,VIR,250-1500,0.000000


In [39]:
mang_stats_df_long = mang_stats_df.melt(id_vars='Country_Code')
mang_stats_df_long = mang_stats_df_long[~mang_stats_df_long.Country_Code.isna()]
mang_stats_df_long['variable'] = mang_stats_df_long['variable'].str.replace('2020_agb_', '').str.replace('tot', 'total')
mang_stats_df_long

,Country_Code,variable,value
0,AGO,total,3.710737e+06
1,AIA,total,1.530619e+01
2,ARE,total,5.707837e+04
3,BHR,total,6.241641e+02
4,BHS,total,1.788835e+06
...,...,...,...
239,TON,avg,9.093408e+01
240,TTO,avg,8.412020e+01
241,VGB,avg,9.491022e+00
242,VIR,avg,2.398343e+01


In [40]:
mang_combined_df = mang_df_long.append(mang_stats_df_long)

In [41]:
mang_combined_df.groupby(['Country_Code', 'variable']).sum().head(21)

value
Country_Code variable              
ABW          0-50      7.787879e-01
             100-150   0.000000e+00
             150-250   0.000000e+00
             250-1500  0.000000e+00
             50-100    2.212121e-01
             avg       2.326628e+01
             total     1.023365e+03
AGO          0-50      3.710584e-01
             100-150   1.148138e-01
             150-250   1.016201e-01
             250-1500  1.800557e-01
             50-100    2.324521e-01
             avg       1.320985e+02
             total     3.710737e+06
AIA          0-50      1.000000e+00
             100-150   0.000000e+00
             150-250   0.000000e+00
             250-1500  0.000000e+00
             50-100    0.000000e+00
             avg       4.705687e+00
             total     1.530619e+01

## Add (staging) locations

In [42]:
locations = pd.read_csv('https://storage.googleapis.com/mangrove_atlas/widget_data/gmw_staging_locations.csv')
locations = locations[['id', 'name', 'location_type', 'iso', 'location_id']]
locations

,id,name,location_type,iso,location_id
0,1561,Worldwide,worldwide,WORLDWIDE,worldwide
1,1560,Estuaire du fleuve Sinnamary,wdpa,GUF,2_0000000000000000084e
2,1559,La Vasière des Badamiers,wdpa,MYT,2_000000000000000005bf
3,1558,Het Pekelmeer,wdpa,BES,2_000000000000000002d2
4,1557,Het Lac,wdpa,BES,2_000000000000000002d1
...,...,...,...,...,...
258,1303,Comoros,country,COM,1_2_2
259,1302,Cameroon,country,CMR,1_2_1
260,1301,Cote d'Ivoire,country,CIV,1_2_0
261,1300,Saloum Delta,aoi,SEN,1_1_2_00000000000000000000


In [43]:
mang_df_final = mang_combined_df.merge(locations[locations['location_type']== 'country'][['iso', 'id']], left_on='Country_Code', right_on='iso', how='left')
mang_df_final = mang_df_final[mang_df_final.id.notna()]
mang_df_final

,Country_Code,variable,value,iso,id
0,AGO,0-50,0.371058,AGO,1398.0
2,ARE,0-50,1.000000,ARE,1369.0
3,BHR,0-50,0.883077,BHR,1374.0
4,BHS,0-50,1.000000,BHS,1375.0
5,ERI,0-50,0.582670,ERI,1310.0
...,...,...,...,...,...
849,TON,avg,90.934079,TON,1321.0
850,TTO,avg,84.120200,TTO,1393.0
851,VGB,avg,9.491022,VGB,1363.0
852,VIR,avg,23.983434,VIR,1397.0


## Final format and save

In [44]:
mang_df_final = mang_df_final[['id', 'variable', 'value']]
mang_df_final.rename(columns={'id':'location_id', 'variable': 'indicator'}, inplace=True)
mang_df_final['year'] = 2020
mang_df_final.head(10)

/Users/angel/opt/anaconda3/envs/py39/lib/python3.9/site-packages/pandas/core/frame.py:5039: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().rename(
/var/folders/wf/_wlxc6cn5js4hh3j7j6x28f80000gn/T/ipykernel_7061/3999650482.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  mang_df_final['year'] = 2020


,location_id,indicator,value,year
0,1398.0,0-50,0.371058,2020
2,1369.0,0-50,1.000000,2020
3,1374.0,0-50,0.883077,2020
4,1375.0,0-50,1.000000,2020
5,1310.0,0-50,0.582670,2020
6,1311.0,0-50,0.000000,2020
7,1376.0,0-50,0.891699,2020
8,1378.0,0-50,0.000000,2020
9,1352.0,0-50,0.000000,2020
10,1353.0,0-50,0.796209,2020


In [45]:
mang_df_final.tail(5)

,location_id,indicator,value,year
849,1321.0,avg,90.934079,2020
850,1393.0,avg,84.120200,2020
851,1363.0,avg,9.491022,2020
852,1397.0,avg,23.983434,2020
853,1367.0,avg,143.940555,2020


In [46]:
mang_df_final.indicator.value_counts()

0-50        101
50-100      101
100-150     101
150-250     101
250-1500    101
total       101
avg         101
Name: indicator, dtype: int64

In [47]:
mang_df_final.to_csv('../../../../data/biomass_widget_data_v2.csv', index=False)